In [1]:
![](assets/mesh_reconstruction.mov)

zsh:1: number expected


In [ ]:
%pip install libigl
%pip install git+https://github.com/skoch9/meshplot.git
%pip install ipywidgets
%pip install pythreejs
%pip install scikit-learn

In [ ]:
import numpy as np
import igl
import meshplot


In [ ]:
from IPython.display import HTML, IFrame, display
meshplot.offline()
class HTML_Plotter(): 
    def plot(self, data, faces=None, c=None, shading=None): 
        p = meshplot.plot(data, faces, c=c, shading=shading, return_plot=True)
        html = p.to_html(imports=True, html_frame=True)
        display(HTML(html))
        return p
        # p.save("temp.html")
        # display(IFrame(src="./temp.html", width=1500, height=1500))
mp = HTML_Plotter()

# Constants

## Miscallenous

In [ ]:
def fprint(arr): 
    with np.printoptions(threshold=np.inf): print(arr)

In [ ]:
floating_point_precision = 1e-8
zero_result_tolerance = 1e-6 # the tolerance for the zero result of the function, which will determine if the point is in the function 
large_positive_number = 1.0

## wendland constant and resolution

In [ ]:
cat_wendland = 0.15
luigi_wendland = 0.08
wendland_factor = cat_wendland
def wendlandFactor(k): 
    global wendland_factor
    return wendland_factor

In [ ]:
cat_resolution = 20
luigi_resolution = 12
resolution = cat_resolution

In [ ]:
def setup_cat(): 
    global wendland_factor, resolution
    wendland_factor = cat_wendland
    resolution = cat_resolution
def setup_luigi(): 
    global wendland_factor, resolution
    wendland_factor = luigi_wendland
    resolution = luigi_resolution

In [ ]:
def wendlandRadius(k, pcbbox_diag): 
    return wendlandFactor(k) * pcbbox_diag

## mesh

In [ ]:
def get_v_f_ni(name_of_mesh):
    v, f = igl.read_triangle_mesh(name_of_mesh)
    v /= 10
    ni = igl.per_vertex_normals(v, f)
    return v, f, ni

## MLS constants

In [ ]:
def polynomialTerms(k):
    return 1 if k == 0 else (4 if k == 1 else 10)

## boundary box conditions

In [ ]:
# how much of the tight bounding box diagonal should the bounding box be enlarged by
bounds_enlargement_factor = 0.05

#how much of the tight bounding box diagonal should the initial epsilon be
initial_eps_factor = 0.01

# to ensure offsetted points are within the enlarged bounding box, we need the initial epsilon factor to be less than the bounds enlargement factor
assert(initial_eps_factor < bounds_enlargement_factor)

In [ ]:
def get_initial_eps(bounds):
    _, _, pcbbox_diag, _ = bounds
    return initial_eps_factor * pcbbox_diag

## Spatial Hashing Constant 

Question: Should grid_spacing be constant or variable depending on resolution? 

previous implemenation: grid_spacing = pcbbox_diag // resolution

In [ ]:
cells_in_spatial_grid = 10

In [ ]:
def mesh_bounds(vertices):
    # initial tight constraint
    pcbbox_min = np.min(vertices, axis=0)
    pcbbox_max = np.max(vertices, axis=0)
    pcbbox_diag = np.linalg.norm(pcbbox_max - pcbbox_min)
    
    # enlarge the bounding box 
    pcbbox_min -= bounds_enlargement_factor * pcbbox_diag
    pcbbox_max += bounds_enlargement_factor * pcbbox_diag
    pcbbox_diag = np.linalg.norm(pcbbox_max - pcbbox_min)

    grid_spacing = (pcbbox_max - pcbbox_min) / cells_in_spatial_grid
    return pcbbox_min, pcbbox_max, pcbbox_diag, grid_spacing

# Tet Grid (Provided)

In [ ]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Step 1 : Setting up the Constraints

### Helpers

In [ ]:
import copy

In [ ]:
regShading = {"point_size": 1,"width": 800, "height": 800}
def shadingPoints(pointsize):
    return {"point_size": pointsize,"width": 800, "height": 800}

In [ ]:
def vectors_length(vectors):
    return np.linalg.norm(vectors, axis=1)

In [ ]:
def find_closest_point(point, points): 
    # vector of distances 
    differences = points - point 
    distances =  vectors_length(differences)
    
    # if zero, set it to infinity - exclude the point itself 
    distances[np.where(np.abs(distances) < floating_point_precision)] = np.inf 
    
    # find the index of the closest by using argmin 
    closest_index = np.argmin(distances)
    return points[closest_index]

In [ ]:
def addPoint(point, points): 
    if points.size == 0: return np.array([point])
    return np.append(points, [point], axis=0)

### Getting a global correct epsilon, then getting the total points with offsetted points

In [ ]:
def get_eps(v, ni, initialEps):
    # the ordering of totalVertices is [v, v+eps*ni, v-eps*ni]
    n = len(v)
    eps = initialEps 
    totalVertices = copy.deepcopy(v)
    
    # get epsilon from a single offset type -- piPlus or piMinus
    # get_offset_point is a function that takes a point and epsilon normal and spits out an updated offset point
    def get_eps_partially(v, ni, eps, totalVertices, get_offset_point):
        for i in range(n): 
            pt = v[i]
            norm = ni[i]
            epsNormal = eps * norm
            pi = get_offset_point(pt, epsNormal)
            totalVertices = addPoint(pi, totalVertices)
            while not np.array_equal(find_closest_point(pi, totalVertices), pt): 
                totalVertices = totalVertices[:-1]
                eps /= 2
                pi = get_offset_point(pt, eps * norm)
                totalVertices = addPoint(pi, totalVertices)
        return eps, totalVertices
    
    # get eps from both piPlus and piMinus
    eps, totalVertices  = get_eps_partially(v, ni, eps, totalVertices, 
                                            get_offset_point=lambda pt, epsNormal: pt + epsNormal)
    eps, _              = get_eps_partially(v, ni, eps, totalVertices, 
                                            get_offset_point=lambda pt, epsNormal: pt - epsNormal)
    return eps

In [ ]:
def get_total_vertices(v, ni, eps):
    epsPos = np.array([])
    epsNeg = np.array([])
    totalVertices = copy.deepcopy(v)

    # now do a single run for the new eps, to get the epsPos and epsNeg
    for i in range(len(v)): 
        pt = v[i]
        norm = ni[i]
        epsNormal = eps * norm
        piPlus = pt + epsNormal
        piMinus = pt - epsNormal
        
        epsPos = addPoint(piPlus, epsPos)
        epsNeg = addPoint(piMinus, epsNeg)
        
    # totalvertices being v + epsPos + epsNeg
    totalVertices = copy.deepcopy(v)
    totalVertices = np.append(totalVertices, epsPos, axis=0)
    totalVertices = np.append(totalVertices, epsNeg, axis=0)
    return totalVertices

### Getting colors 

In [ ]:
def getRGB_colors(total_points_size):
    colors = np.zeros((total_points_size,3))
    one_third = total_points_size // 3
    colors[:one_third,0] = 1
    colors[one_third:2*one_third,1] = 1
    colors[2*one_third:,2] = 1
    return colors

### Plotting with the offsetted points

In [ ]:
def plot_with_offsetted_points(v, ni, pointSize):
    bounds = mesh_bounds(v)
    initialEps = get_initial_eps(bounds)
    eps = get_eps(v, ni, initialEps)
    totalVertices = get_total_vertices(v, ni, eps) 
    colors = getRGB_colors(len(totalVertices))
    return mp.plot(totalVertices, c=colors, shading=shadingPoints(pointSize))

In [ ]:
def plot_mesh_with_offsetted_points(name_of_mesh, pointSize=4): 
    v, _, ni = get_v_f_ni(name_of_mesh)
    return plot_with_offsetted_points(v, ni, pointSize)

### Required Output

 * Plot of the provided point cloud shaded with green, blue, and red dots.

Green is outside, red is on the surface, and blue is inside.

In [ ]:
meshplot.offline()

In [ ]:
setup_luigi()
plot_mesh_with_offsetted_points("data/luigi.off", pointSize=1)
setup_cat()

In [ ]:
plot_mesh_with_offsetted_points("data/cat.off")

Question: why does it look so close? 

Answer from Professor: It depends on your epsilon. You can choose a smaller and bigger beginnning epsilon.


# Step 2: Use MLS interpolation to extend to function `f`

### Create a grid sampling the 3D space

This is taken care of within the plotting function

### Closest points

 **Important**: explicitty write a function `closest_points(point, points, h)` that retreives the indices all points in `points` that are at distance less than `h` from `point`.

In [ ]:
def closest_points(point, points, h): 
    # filter the points that are within the radius
    differences = points - point
    distances =  vectors_length(differences)
    indices = np.where(distances < h)
    
    # `indices` is a tuple, so we need to get the first element, which is the array of indices
    return indices[0]

### Experiment with Wendland Radius using closestpoints 

Goal: Get a correct wendland radius where, for all points, the number of closest points to all the points is more or equal to twice the number of polynomial terms.

Results: (This was run on the 'cat' point cloud then cross checked with the 'luigi' point cloud)

Let the left expression be the wendland radius experimented on.
Let conditions on the right be whether or not there are 2 * polynomial terms closest points to the point at the given radius.

**k = 0**

0.004 * pcbbox -> False

0.005 * pcbbox -> True

So, I tried 0.006 * pcbbox

**k = 1**

0.08 * pcbbox -> False

0.09 * pcbbox -> True

So, I tried 0.1 * pcbbox

**k = 2**

0.12 * pcbbox -> False 

0.13 * pcbbox -> True

So, I tried 0.14 * pcbbox 


In the end, I used 0.15 as the factor, and you can see this result in the [Step 5 Section](#step-5-extracting-the-surface)

In [ ]:
def experiment_with_wendland(experimentalWendlandFactor, experimentalK, points, ni):
    bounds = mesh_bounds(points)
    initialEps = get_initial_eps(bounds)
    _, _, pcbbox_diag, _ = bounds
    experimentalRadius = experimentalWendlandFactor * pcbbox_diag
    eps = get_eps(points, ni, initialEps)
    totalVertices = get_total_vertices(points, ni, eps) 
    
    def experimentClosestPoints(point): 
        # check how many points are within the radius
        indices = closest_points(point, totalVertices, experimentalRadius)
        return len(indices)
    a = np.apply_along_axis(experimentClosestPoints, axis=1, arr=totalVertices)
    return (a > 2 * polynomialTerms(experimentalK)).all()

def experiment_with_wendland_on_mesh(name_of_mesh, experimentalWendlandFactor, experimentalK):
    v, _, ni = get_v_f_ni(name_of_mesh)
    return experiment_with_wendland(experimentalWendlandFactor, experimentalK, v, ni)

In [ ]:
experiment_with_wendland_on_mesh("data/cat.off", experimentalWendlandFactor=0.14, experimentalK=2)

### MLS Interpolation

#### get wendland matrix 

In [ ]:
def wendland(point, radius): 
    d = np.linalg.norm(point)
    if d >= radius: return 0
    return ((1 - d / radius) ** 4) * (4 * d / radius + 1)

In [ ]:
# get wendland matrix 
def W(x, radius, closeByPoints, total_points): 
    # compute diagonal matrix
    n = total_points.shape[0]
    indices = closeByPoints
    values = np.zeros((n))
    for i in indices: 
        values[i] = wendland(x - total_points[i], radius)
    diagonal = np.diag(values)
    return diagonal

#### get Basis Matrix

In [ ]:
def B_k(total_points, k): 
    n = total_points.shape[0]
    ones = np.ones((n, 1))
    if k == 0: return ones
    
    x, y, z = total_points[:,0], total_points[:,1], total_points[:,2]
    if k == 1: 
        B = np.empty((0,4))
        for i in range(n): B = np.vstack([B, [1, x[i], y[i], z[i]]])
        return B
    
    xy, xz, yz = x * y, x * z, y * z
    xx, yy, zz = x ** 2, y ** 2, z ** 2
    if k == 2: 
        B = np.empty((0,10))
        for i in range(n): B = np.vstack([B, [1, x[i], y[i], z[i], xy[i], xz[i], yz[i], xx[i], yy[i], zz[i]]])
        return B
    
    return None

#### get `d`

In [ ]:
# first 1/3 are 0, next 1/3 are eps, last 1/3 are -eps
def get_d(total_points, eps): 
    d = np.zeros((total_points.shape[0]))
    one_third = total_points.shape[0] // 3
    d[one_third:2*one_third] = eps
    d[2*one_third:] = -eps
    return d

#### linear solver

In [ ]:
def BtWxB(B, BtWx): 
    return BtWx @ B

def BtWxd(BtWx, d): 
    return BtWx @ d

In [ ]:
def solveA(x, total_points, radius, basis, d, closeByPoints): 
    B = basis
    W_matrix = W(x, radius, closeByPoints, total_points)
    BtWx = B.T @ W_matrix
    
    A = BtWxB(B, BtWx)
    b = BtWxd(BtWx, d)
    return np.linalg.solve(A, b)

#### Get weights

In [ ]:
def getFxValue(pt, weights, k):
    one = 1 
    if k == 0: 
        aone = weights[0]
        return aone * one
    
    x, y, z = pt[0], pt[1], pt[2]
    if k == 1: 
        aone, ax, ay, az = weights
        return aone * one + ax * x + ay * y + az * z
    
    xy, xz, yz,xsq, ysq, zsq = x * y, x * z, y * z, x ** 2, y ** 2, z ** 2
    if k == 2:
        aone, ax, ay, az, axy, axz, ayz, axx, ayy, azz = weights
        return (aone * one + ax * x + ay * y + az * z + axy * xy + axz * xz + ayz * yz + axx * xsq + ayy * ysq + azz * zsq)
    
    return None

### MLS Final - Solving for `f`

#### MLS

In [ ]:
def MLS(pt, total_points, radius, basis, d, k): 
    # check that there are enough points around it to make a MLS fit 
    closeByPoints = closest_points(pt, total_points, radius)
    if len(closeByPoints) < 2 * polynomialTerms(k): 
        return large_positive_number # return a large number to indicate that the point is not outside the surface
    
    # first get coefficients
    a = solveA(pt, total_points, radius, basis, d, closeByPoints)
    
    # solve for the value 
    value = getFxValue(pt, a, k)
    return value

#### `f` for all points 

In [ ]:
def plot_implicit_function(total_points, eps, bounds, k=2, plot=True, pointsSize=4): 
    pcbbox_min, pcbbox_max, pcbbox_diag, _ = bounds
    x, T = tet_grid((resolution,resolution,resolution), 
                pcbbox_min - 0.05 * pcbbox_diag, pcbbox_max + 0.05 * pcbbox_diag)
    wendland_radius = wendlandRadius(k, pcbbox_diag)
    basis, d = B_k(total_points, k), get_d(total_points, eps)
    indicators = np.apply_along_axis(
        lambda point: MLS(point, total_points, wendland_radius, basis, d, k), 
        axis=1, arr=x)
    
    colors = np.zeros_like(indicators)
    colors[indicators >= 0] = 1
    colors[indicators < 0] = -1
    if plot: mp.plot(x, c=colors, shading={"point_size": pointsSize,"width": 800, "height": 800})
    else: return x, T, indicators 

#### Plotting Functions

In [ ]:
# only plot points - no extraction yet
def plot_mesh_points(name_of_mesh, k=2, plot=True, pointsSize=4): 
    v, _, ni = get_v_f_ni(name_of_mesh)
    bounds = mesh_bounds(v)
    initialEps = get_initial_eps(bounds)
    eps = get_eps(v, ni, initialEps)
    total_points = get_total_vertices(v, ni, eps)
    return plot_implicit_function(total_points, eps, bounds, k, plot, pointsSize)

### Required Output of Section

* Plot of the grid points x colored according of being inside or outside the input cloud.

Discussion: There are some noise/artifacts around the point cloud, but the shape of the point cloud is captured. As `k` increases, it seems there are more artifacts. 

In [ ]:
plot_mesh_points("data/cat.off", k=0)

In [ ]:
plot_mesh_points("data/cat.off", k=1)

In [ ]:
plot_mesh_points("data/cat.off", k=2)

# Step 3: Implementing a spatial index to accelerate neighbor calculations

There is no required output of this section. Refer to [this section](#additional-reports) for evidence of the spatial acceleration data structure.

## Create a Hash of all points and place points into the bin

In [ ]:
# grid spacing should be also [x,y,z] representing the spacing for each dimension
def get_spatial_index(point, pcbbox_min, grid_spacing): 
    return ((point - pcbbox_min) // grid_spacing).astype(int)


In [ ]:
def add_to_spatial_hash(point, associated_index, spatial_hash, bounds): 
    pcbbox_min, _, _, grid_spacing = bounds
    x, y, z = get_spatial_index(point, pcbbox_min, grid_spacing)
    spatial_hash[x][y][z].append(associated_index)
    return spatial_hash

In [ ]:
# a hash of the indices of the points
def get_spatial_grid(points, bounds):
    x_s, y_s, z_s = cells_in_spatial_grid, cells_in_spatial_grid, cells_in_spatial_grid
    spatial_hash = np.empty((x_s, y_s, z_s, 0)).tolist()
    for index in range(len(points)): 
        add_to_spatial_hash(points[index], index, spatial_hash, bounds)
    return spatial_hash

In [ ]:
# removes the last element in the spatial bin where the point is located
def remove_last_from_spatial_hash(point, spatial_hash, bounds): 
    pcbbox_min, _, _, grid_spacing = bounds
    x, y, z = get_spatial_index(point, pcbbox_min, grid_spacing)
    spatial_hash[x][y][z] = spatial_hash[x][y][z][:-1]
    return spatial_hash

## Get the set of points within a certain distance

In [ ]:
def get_filtered_indices(point, h, spatial_hash, bounds): 
    # get the min and max cells 
    # you have to add 1 to the max cell because you want to range from min to max inclusive
    pcbbox_min, _, _, grid_spacing = bounds
    p_h_min, p_h_max = get_spatial_index(point - h, pcbbox_min, grid_spacing), get_spatial_index(point + h, pcbbox_min, grid_spacing) + 1
    xmin, ymin, zmin = p_h_min
    xmax, ymax, zmax = p_h_max
    
    # clamp cell values to be within the grid
    x_s, y_s, z_s = cells_in_spatial_grid, cells_in_spatial_grid, cells_in_spatial_grid
    xmin, ymin, zmin, xmax, ymax, zmax = max(0, xmin), max(0, ymin), max(0, zmin), min(x_s, xmax), min(y_s, ymax), min(z_s, zmax) 
    
    # get as indices, using list comprehension
    filtered_indices =  [element 
                                for x in range(xmin, xmax) 
                                for y in range(ymin, ymax)
                                for z in range(zmin, zmax)
                                for element in spatial_hash[x][y][z]]
    return np.array(filtered_indices)

## Optimization for `find_closest_point` and `get_eps`

Note: I did not change `find_closest_point` but rather `get_eps` when finding the closest point. TA said this is okay.

In [ ]:
def find_closest_point_accelerated(point, points, epsNorm=None, spatial_hash=None, bounds=None):
    accelerated_mode = spatial_hash is not None and bounds is not None and epsNorm is not None
    if accelerated_mode: 
        # enlarge eps a bit 
        # since we just added point, which is epsNorm away from the original point, there is at least one point within the l2 distance of epsNorm
        epsNorm *= 1.01
        radial_distance = np.linalg.norm(epsNorm)
        filtered_indices = get_filtered_indices(point, radial_distance, spatial_hash, bounds)
        points = points[filtered_indices]
    
    # vector of distances 
    differences = points - point 
    distances =  vectors_length(differences)
    
    # if zero, set it to infinity - exclude the point itself 
    distances[np.where(np.abs(distances) < floating_point_precision)] = np.inf 
    
    # find the index of the closest by using argmin 
    closest_index = np.argmin(distances)
    
    return points[closest_index]    

In [ ]:
# in addition to before, we add offsetted points into the spatial hash 
# and we also call an accelerated find closest points 
def get_eps_acccelerated(v, ni, initialEps, bounds, initial_spatial_hash):
    # the ordering of totalVertices is [v, v+eps*ni, v-eps*ni]
    n = len(v)
    eps = initialEps 
    totalVertices = copy.deepcopy(v)
    spatial_hash = initial_spatial_hash
    
    # get epsilon from a single offset type -- piPlus or piMinus
    # get_offset_point is a function that takes a point and epsilon normal and spits out an updated offset point
    # get_associated_index is a function that takes an original index, and returns the associated index in the totalVertices
    def get_eps_partially(v, ni, eps, totalVertices, bounds, spatial_hash, get_offset_point, get_associated_index):
        for i in range(len(v)): 
            pt = v[i]
            norm = ni[i]
            epsNormal = eps * norm
            pi = get_offset_point(pt, epsNormal)
            
            totalVertices = addPoint(pi, totalVertices)
            associated_index = get_associated_index(i)
            spatial_hash = add_to_spatial_hash(pi, associated_index, spatial_hash, bounds)
            while not np.array_equal(find_closest_point(pi, totalVertices), pt): 
                # remove the old point 
                totalVertices = totalVertices[:-1]
                spatial_hash = remove_last_from_spatial_hash(pi, spatial_hash, bounds)
                
                # get the new point
                eps /= 2
                pi = get_offset_point(pt, eps * norm)
                
                # add the new point
                totalVertices = addPoint(pi, totalVertices)
                associated_index = get_associated_index(i)
                spatial_hash = add_to_spatial_hash(pi, associated_index, spatial_hash, bounds)
        return eps, totalVertices
    
    # get eps from both piPlus and piMinus
    eps, totalVertices  = get_eps_partially(v, ni, eps, totalVertices, bounds, spatial_hash, 
                                            get_offset_point=    lambda pt, epsNormal: pt + epsNormal, 
                                            get_associated_index=lambda i: i + n)
    eps, _              = get_eps_partially(v, ni, eps, totalVertices, bounds, spatial_hash,
                                            get_offset_point=    lambda pt, epsNormal: pt - epsNormal,
                                            get_associated_index=lambda i: i + 2 * n)
    return eps, spatial_hash

## Optimization for `closest_point` and `MLS`

### Closest Points 

In [ ]:
def closest_points_accelerated(point, points, h, bounds, spatial_hash): 
    #  get the filtered points within cells of the spatial hash within the radius
    filtered_indices = get_filtered_indices(point, h, spatial_hash, bounds)
    if filtered_indices.shape[0] == 0: return np.array([])
    points = points[filtered_indices]
    
    # further filtering
    # filter the points that are within the radius
    differences = points - point
    distances =  vectors_length(differences)
    indices = np.where(distances < h)
    
    # remap the indices to the original indices
    indices = filtered_indices[indices]
    return indices

### Passing of variables down

In [ ]:
def MLS_accelerated(pt, total_points, radius, basis, d, bounds, spatial_hash, k): 
    # check that there are enough points around it to make a MLS fit 
    closeByPoints = closest_points_accelerated(pt, total_points, radius, bounds, spatial_hash)
    if len(closeByPoints) < 2 * polynomialTerms(k): 
        return large_positive_number # return a large number to indicate that the point is not outside the surface
    
    # first get coefficients
    a = solveA(pt, total_points, radius, basis, d, closeByPoints)
    
    # solve for the value 
    value = getFxValue(pt, a, k)
    return value

In [ ]:
def plot_implicit_function_accelerated(total_points, eps, bounds, spatial_hash, k=2, plot=True, pointsSize=4):
    # all the same except for the addition of spatial_hash
    pcbbox_min, pcbbox_max, pcbbox_diag, _ = bounds
    x, T = tet_grid((resolution,resolution,resolution), 
                pcbbox_min - 0.05 * pcbbox_diag, pcbbox_max + 0.05 * pcbbox_diag)
    wendland_radius = wendlandRadius(k, pcbbox_diag)
    basis, d = B_k(total_points, k), get_d(total_points, eps)
    indicators = np.apply_along_axis(
        lambda point: MLS_accelerated(point, total_points, wendland_radius, basis, d, bounds, spatial_hash, k), 
        axis=1, arr=x)
    
    colors = np.zeros_like(indicators)
    colors[indicators >= 0] = 1
    colors[indicators < 0] = -1
    if plot: mp.plot(x, c=colors, shading={"point_size": pointsSize,"width": 800, "height": 800})
    else: return x, T, indicators 

In [ ]:
# only plot points - no extraction yet
def plot_mesh_points_accelerated(name_of_mesh, k=2, plot=True, pointsSize=4): 
    v, _, ni = get_v_f_ni(name_of_mesh)
    bounds = mesh_bounds(v)
    initialEps = get_initial_eps(bounds)
    spatial_hash = get_spatial_grid(v, bounds)
    eps, spatial_hash = get_eps_acccelerated(v, ni, initialEps, bounds, initial_spatial_hash=spatial_hash)
    total_points = get_total_vertices(v, ni, eps)
    return plot_implicit_function_accelerated(total_points, eps, bounds, spatial_hash, k, plot, pointsSize)

# Step 4: Using a non-axis-aligned grid 

### explanation of pca

I never learned about PCA before, so this was an example I tried to really understand it. Accordingly, I applied the 2D understanding of it to a point cloud.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
# how PCA works
# here's the plotting as well 
values = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
transformedValues = pca.fit_transform(values)

the blue are the original points, and the orange are the transformed points. It makes the transformed points have the highest variance on the x axis, which is going to be what we want for the bounding box of the point cloud

In [ ]:
plt.scatter(values[:, 0], values[:, 1])
plt.scatter(transformedValues[:, 0], transformedValues[:, 1])

### apply PCA transform

I directly apply it to the vertices right when reading them from the mesh. This ensures the normals are also transformed.


In [ ]:
def apply_pca_transform(points): 
    pca = PCA(n_components=3)
    return pca.fit_transform(points)

In [ ]:
# same as before, except with PCA applied
def get_v_f_ni(name_of_mesh):
    v, f = igl.read_triangle_mesh(name_of_mesh)
    v /= 10
    v = apply_pca_transform(v)
    ni = igl.per_vertex_normals(v, f)
    return v, f, ni

### test it using just points

clearly, the luigi points are aligned. The next section shows this in the grid, which more clearly shows that it is aligned as the bounds are minimal and axis aligned. 

In [ ]:
v, f, ni = get_v_f_ni("data/luigi.off")
mp.plot(v, shading={"point_size": 5,"width": 800, "height": 800})

## Required Output

Plot of the grid with nodes colored according to their implicit function values. There is not a noticable speedup, but the next sub-section talks about the speedup. Here, at least, you can see that the results are the same visually. Please change `resolution` under the `Spatial Hashing Constants` section if it loads too slow.

In [ ]:
plot_mesh_points("data/cat.off", k=2)

In [ ]:
plot_mesh_points_accelerated("data/cat.off", k=2)

At this point, I recommend changing resolution to 10, so it does not take too long. `10` will give `1 minute` on my computer. But any more than that makes it much much longer than 5 minutes. Refer to [this section](#boundary-box-conditions) to change it. There is a small speedup for luigi.off by a few seconds only. For this low resolution, a lower wendland radius was also found to be better. 

In [ ]:
setup_luigi()
plot_mesh_points("data/luigi.off", k=1, pointsSize=1)

In [ ]:
plot_mesh_points_accelerated("data/luigi.off", k=1, pointsSize=1)
setup_cat()

It is clear that the luigi mesh is now aligned. 

# Step 5: Extracting the surface

### Functions to extract the surface 

In [ ]:
def get_max_connected_component(surface_f): 
    components = igl.facet_components(surface_f)
    max_freq_component = np.argmax(np.bincount(components))
    max_component = surface_f[components == max_freq_component]
    return max_component

In [ ]:
def plot_extracted_surface(name_of_mesh, k=2): 
    x, T, indicators = plot_mesh_points_accelerated(name_of_mesh, k, False)
    sv, sf, _, _ = igl.marching_tets(x, T, indicators, 0)
    max_component = get_max_connected_component(sf)
    mp.plot(sv, max_component, shading={"point_size": 5,"width": 800, "height": 800})

### Required Outputs

At this point, bringing up the resolution is a good idea. Maybe 20 to 30 for the cat and 10 for the luigi.

We now perform on `cat.off` for the experiments, and report results.

**NOTE**: I am unable to get a proper surface out of luigi, perhaps because the resolution is too low. It takes infinitely long for larger resolutions, so I am unable to test. However, the `mesh_points` give a decent idea of the surface that proves my logic is probably not wrong. You can find this in Step 4's [Required Output](#required-output) section. For this reason, I am also unable to properly test a good wendland radius/factor for luigi.

In [ ]:
setup_cat()
plot_extracted_surface("data/cat.off", k=0)

The general shape of the mesh is there. It doesn't really look like Luigi, but that is expected because the resolution is low. I wasn't able to run it on a higher resolution due to time constraints and my computer not able to handle more than a few minutes before something goes wrong.

In [ ]:
prev_resolution = resolution
previous_wendland_factor = wendlandFactor(1)
def wendlandFactor(k): return 0.1
resolution = 12
plot_extracted_surface("data/luigi.off", k=2)
resolution = prev_resolution # change it back
def wendlandFactor(k): return previous_wendland_factor

### Experimenting with `k` (polynomial terms)

k = 0, 1, 2

Results: When `k=0`, the surface is not fully captured. However, it is much more smooth-looking. As `k` increases, the surface is more captured, but there are more artifacts and oscillations, which is expected since the polynomial used to fit is more complex.

In [ ]:
plot_extracted_surface("data/cat.off", k=0)

In [ ]:
plot_extracted_surface("data/cat.off", k=1)

In [ ]:
plot_extracted_surface("data/cat.off", k=2)

### Experimenting with wendland radius

First refer to [this section](#experiment-with-wendland-radius-using-closestpoints) to see the initial experiments done with some logic. Here, I do more random experiments on the wendland radius with three different values (15%, 20%, 30%) for the sake of seeing the differences fully qualitatively.

Results: The higher the wendland factor, the more artifacts and oscillations. The lower the wendland factor, the less the surface is captured. 0.2 seemed to work fine with the cat on a qualitative basis, so that is what I kept. 

In [ ]:
def wendlandFactor(k): return 0.3
plot_extracted_surface("data/cat.off", k=1)

In [ ]:
def wendlandFactor(k): return 0.20
plot_extracted_surface("data/cat.off", k=1)

In [ ]:
def wendlandFactor(k): return 0.15
plot_extracted_surface("data/cat.off", k=1)

choose 0.2

In [ ]:
def wendlandFactor(k): return 0.2

### Experimenting with grid resolutions

#### Function for anisotropic resolutions

The idea for anisotropic resolutions, as the TA explained, is that you fix one of the dimension's resolutions. Then, you can change the other two dimensions' resolutions to have the same length. So you end up with cubic voxel resolutions.

In [ ]:
def get_anisotropic_resolutions(bounds, fixedX=None, fixedY=None, fixedZ=None):
    x_resolution, y_resolution, z_resolution = fixedX, fixedY, fixedZ
    pcbbox_min, pcbbox_max, _, _ = bounds
    x_dist, y_dist, z_dist = pcbbox_max - pcbbox_min
    if x_resolution is not None: 
        fixed_voxel_length = x_dist / fixedX
        y_resolution = y_dist // fixed_voxel_length
        z_resolution = z_dist // fixed_voxel_length
    elif y_resolution is not None:
        fixed_voxel_length = y_dist / fixedY
        x_resolution = x_dist // fixed_voxel_length
        z_resolution = z_dist // fixed_voxel_length
    elif z_resolution is not None:
        fixed_voxel_length = z_dist / fixedZ
        x_resolution = x_dist // fixed_voxel_length
        y_resolution = y_dist // fixed_voxel_length
    return int(x_resolution), int(y_resolution), int(z_resolution)

In [ ]:
def plot_implicit_function_accelerated(total_points, eps, bounds, spatial_hash, k=2, plot=True, pointsSize=4):
    # all the same except for the addition of spatial_hash
    pcbbox_min, pcbbox_max, pcbbox_diag, _ = bounds
    x_resolution, y_resolution, z_resolution = get_resolution(bounds)
    x, T = tet_grid((x_resolution, y_resolution, z_resolution), 
                pcbbox_min - 0.05 * pcbbox_diag, pcbbox_max + 0.05 * pcbbox_diag)
    wendland_radius = wendlandRadius(k, pcbbox_diag)
    basis, d = B_k(total_points, k), get_d(total_points, eps)
    indicators = np.apply_along_axis(
        lambda point: MLS_accelerated(point, total_points, wendland_radius, basis, d, bounds, spatial_hash, k), 
        axis=1, arr=x)
    
    colors = np.zeros_like(indicators)
    colors[indicators >= 0] = 1
    colors[indicators < 0] = -1
    if plot: mp.plot(x, c=colors, shading={"point_size": pointsSize,"width": 800, "height": 800})
    else: return x, T, indicators 

### Experiment with Anisotropic Resolutions

Results: Anisotropic resolution in the x direction with 20 is not good. But anisotropic with direction in y and z are good, with y being the best looking surface. It depends on how big x y z are of the bounding box it seems. 

In [ ]:
# anisotropic 20 in x 
def get_resolution(bounds):
    return get_anisotropic_resolutions(bounds, fixedX=20)

plot_extracted_surface("data/cat.off", k=2)

In [ ]:
# anisotropic 20 in y
def get_resolution(bounds):
    return get_anisotropic_resolutions(bounds, fixedY=20)

plot_extracted_surface("data/cat.off", k=2)

In [ ]:
# anisotropic 20 in z
def get_resolution(bounds):
    return get_anisotropic_resolutions(bounds, fixedZ=20)

plot_extracted_surface("data/cat.off", k=2)

#### Different Fixed Resolutions

The higher the resolution, the more the surface is captured. However, it takes more time. `resolution=20` is a good balance between time and capturing the surface.

In [ ]:
def plot_implicit_function_accelerated(total_points, eps, bounds, spatial_hash, k=2, plot=True, pointsSize=4):
    # all the same except for the addition of spatial_hash
    pcbbox_min, pcbbox_max, pcbbox_diag, _ = bounds
    x, T = tet_grid((resolution,resolution,resolution), 
                pcbbox_min - 0.05 * pcbbox_diag, pcbbox_max + 0.05 * pcbbox_diag)
    wendland_radius = wendlandRadius(k, pcbbox_diag)
    basis, d = B_k(total_points, k), get_d(total_points, eps)
    indicators = np.apply_along_axis(
        lambda point: MLS_accelerated(point, total_points, wendland_radius, basis, d, bounds, spatial_hash, k), 
        axis=1, arr=x)
    
    colors = np.zeros_like(indicators)
    colors[indicators >= 0] = 1
    colors[indicators < 0] = -1
    if plot: mp.plot(x, c=colors, shading={"point_size": pointsSize,"width": 800, "height": 800})
    else: return x, T, indicators 

# this is the reset of the function to use fixed resolution

In [ ]:
resolution = 10 
plot_extracted_surface("data/cat.off", k=2)

In [ ]:
resolution = 20
plot_extracted_surface("data/cat.off", k=2)

In [ ]:
resolution = 30
plot_extracted_surface("data/cat.off", k=2)

Results: 

Change Resolution Back

In [ ]:
resolution = 20

# Additional Reports

This section is for any code tests and additional reports that I did not include in the main sections.

# Optional Tasks

## Task 1

(2 points) In Interpolating and Approximating Implicit Surfaces from Polygon Soup normals are used differently to define the implicit surface. Instead of generating new sample points offset in the positive and negative normal directions, the paper uses the normal to define a linear function for each point cloud point: the signed distance to the tangent plane at the point. Then the values of these linear functions are interpolated by MLS. Implement Section 3.3 of the paper and append to your report a description of the method and how it compares to the original point-value-based approach. Estimate a normal for results obtained with single dataset.

Description of the method: The idea is that, instead of the `d` vector we have `[zeros * n, epsilon * n, -epsilon * n]` we have only `[sdf(x)]` for `x` being our initial `n` constraint points, and `sdf` being the signed distance function to the tangent plane at the point. We then have to compute `d` for each point we are considering, and we also need information of the normals. Then we simply apply the same MLS logic as before, with a different `d` vector and different set of constraint points. 

Results: As seen after plotting, the plot is similar but a little different. It seems to have some more artifacts but also seems more smooth and no longer as much oscillation. 

"Estimate a normal for results obtained with single dataset." I am not sure what this is asking, but it sounds like it is just an instruction to get the normals of the surface of each point and to get the signed distance of any point to all such tangents based on the normals. I am doing this in the `get_d_locally` step. 

#### Computing SDF

In [ ]:
def compute_signed_distance(point_x, point_i, normal_i): 
    return np.dot(normal_i, point_x - point_i)

In [ ]:
def get_d_locally(point, total_points, normals): 
    d = []
    for i in range(len(total_points)): 
        d.append(compute_signed_distance(point, total_points[i], normals[i]))
    return np.array(d)

#### Passing down the SDF

In [ ]:
def solveA(x, total_points, normals, radius, basis, closeByPoints): 
    B = basis
    W_matrix = W(x, radius, closeByPoints, total_points)
    d_x = get_d_locally(x, total_points, normals)
    BtWx = B.T @ W_matrix
    
    A = BtWxB(B, BtWx)
    b = BtWxd(BtWx, d_x)
    return np.linalg.solve(A, b)

In [ ]:
def MLS(pt, total_points, normals, radius, basis, k): 
    # check that there are enough points around it to make a MLS fit 
    closeByPoints = closest_points(pt, total_points, radius)
    if len(closeByPoints) < 2 * polynomialTerms(k): 
        return large_positive_number # return a large number to indicate that the point is not outside the surface
    
    # first get coefficients
    a = solveA(pt, total_points, normals, radius, basis, closeByPoints)
    
    # solve for the value 
    value = getFxValue(pt, a, k)
    return value

In [ ]:
def plot_implicit_function(total_points, normals, bounds, k=2, plot=True, pointsSize=4): 
    pcbbox_min, pcbbox_max, pcbbox_diag, _ = bounds
    x, T = tet_grid((resolution,resolution,resolution), 
                pcbbox_min - 0.05 * pcbbox_diag, pcbbox_max + 0.05 * pcbbox_diag)
    wendland_radius = wendlandRadius(k, pcbbox_diag)
    basis = B_k(total_points, k)
    indicators = np.apply_along_axis(
        lambda point: MLS(point, total_points, normals, wendland_radius, basis, k), 
        axis=1, arr=x)
    
    colors = np.zeros_like(indicators)
    colors[indicators >= 0] = 1
    colors[indicators < 0] = -1
    if plot: mp.plot(x, c=colors, shading={"point_size": pointsSize,"width": 800, "height": 800})
    else: return x, T, indicators 

#### new way to get total points

In [ ]:
# only plot points - no extraction yet
def plot_mesh_points(name_of_mesh, k=2, plot=True, pointsSize=4): 
    v, _, ni = get_v_f_ni(name_of_mesh)
    bounds = mesh_bounds(v)
    total_points = v
    return plot_implicit_function(total_points, ni, bounds, k, plot, pointsSize)

#### Plot

In [ ]:
def plot_extracted_surface(name_of_mesh, k=2): 
    x, T, indicators = plot_mesh_points(name_of_mesh, k, False)
    sv, sf, _, _ = igl.marching_tets(x, T, indicators, 0)
    max_component = get_max_connected_component(sf)
    mp.plot(sv, max_component, shading={"point_size": 5,"width": 800, "height": 800})

In [ ]:
plot_extracted_surface("data/cat.off", k=1)

## Task 2: Poisson Reconstruction

Resulting Screenshot: ![Image](./poisson_cat.png)

### Compared to mine, it is clear that the Poisson Reconstruction is way more smooth and has much less artifacts, seemingly very robust with noisy artifacts. Even with tuning my parameters, I won't be able to achieve that smoothness with that many features represented, while still having barely any artifacts at all. Furthermore, it was extremely fast--- much faster than my MLS implementation, even when it had tried to do Luigi's mesh. Some similarties I noticed is that you also have to tune parameters. For example, below is a screenshot of Luigi's mesh when not tuned properly for Poisson reconstruction. The hat is not captured correctly. However, an advantage of MLS seems to be that it keeps sharp features, whereas Poisson reconstruction seems to always smooth them out and preserve continuity.

Luigi: ![Image](./poisson_luigi.png)